In [9]:
from __future__ import division
from math import log, exp, factorial
from random import random
from collections import Counter
import numpy as np

In [1]:
LFMEMO = {}
def logfact(n):
    # return ln(n!)
    if n in LFMEMO:
        return LFMEMO[n]
    lf = sum(log(i) for i in range(1, n+1))
    LFMEMO[n] = lf
    return lf

LTMEMO = {}
def logtutte(n, m):
    if n < 0 or m < 0:
        return -float('inf')
    #if (n, m) in LTMEMO:
        #return LTMEMO[(n, m)]
    val = (n+1)*log(2) + logfact(2*m+1) + logfact(2*m+3*n)
    val -= logfact(m)*2 + logfact(n) + logfact(2*m+2*n+2)    
    #LTMEMO[(n, m)] = val
    return val

def logapprox(n, m):
    return logfact(2*m) - 2*logfact(m) - log(m+1) + n*log(13.5) - log(n**2.5)

In [2]:
def rtutte(n, m):
    total = 0
    if n > 0:
        val = tutte(n-1, m+1)
        print('inner', val)
        total += val
    for j in range(0, m):
        for k in range(0, n+1):
            a = tutte(k, j)
            b = tutte(n-k, m-j-1)
            print((k, j), (n-k, m-j-1), a, b, a*b)
            total += a*b
    return total

In [3]:
def logsample(N):
    probs = []
    sizes = []
    for n in range(0, N-1):
        m = N - n - 2
        probs.append(logtutte(n, m))
        sizes.append((n, m))
    high = max(probs)
    probs = [exp(p - high) for p in probs]
    total = sum(probs)
    probs = [p/total for p in probs]
    tot = 0
    for i in range(len(probs)):
        tot += probs[i]
    n, m = sizes[np.random.choice(N-1, p=probs)]
    return n, m

In [4]:
def logmake_graph(G, internal, external):
    n = len(internal)
    m = len(external)-2
    if n == 0 and m == 0:
        # nothing else to add
        return
    logtotal_graphs = logtutte(n, m)
    logadd_internal = logtutte(n-1, m+1)
    if random() < exp(logadd_internal - logtotal_graphs):
        # add internal triangle
        triangle = (external[0], external[1], internal[0])
        G.append(triangle)
        logmake_graph(G, internal[1:], external + [internal[0]])
    else:
        logcounts = []
        choices = []
        for j in range(0, m):
            for k in range(0, n+1):
                logcount = logtutte(k, j) + logtutte(n-k, m-j-1)
                logcounts.append(logcount)
                # there are [count] triangulations with k internal and j+2 external vertices on one side and the rest on the other
                choices.append((k, j))
        high = max(logcounts)
        counts = [exp(c-high) for c in logcounts]
        total = sum(counts)
        counts = [c/total for c in counts]
        choice = choices[np.random.choice(len(choices), p=counts)]
        k, j = choice
        triangle = (external[0], external[1], external[j+2])
        G.append(triangle)
        left = external[1:j+3]
        right = external[j+2:] + [external[0]]
        logmake_graph(G, internal[:k], left)
        logmake_graph(G, internal[k:], right)
    return G

def sample_graph(n, m):
    internal = range(m+2, m+n+2)
    external = range(0, m+2)
    G = []
    return logmake_graph(G, internal, external)

In [46]:
# show that the number of graphs generated by this algorithm is the same(ish) as T(n, m)
from collections import Counter
graphs = Counter()
for _ in range(100000):
    n = 4
    m = 2
    internal = range(m+2, m+n+2)
    external = range(0, m+2)
    G = []
    graphs[tuple(logmake_graph(G, internal, external))] += 1
G = []
G = make_graph(G, internal, external)
print len(graphs), tutte(n, m)

9600 9600.0


In [7]:
# estimating runtime -- assumed n^2*log(n)
COUNT = 0
SIZE = 640
for _ in range(100):
    G = []
    sample_graph(*logsample(SIZE))
print COUNT/100

447862.876


In [28]:
import time
start = time.time()
for _ in range(50):
    x = sample_graph(*logsample(200))
print time.time() - start

13.8935880661
